## 多智能体强化学习--相关笔记&梳理

#### 多智能体存在问题：

1. **维度爆炸：** 状态空间变大，联结动作空间维度非常大，计算复杂

2. **目标奖励确定困难：** _多智能体系统中每个智能体的任务可能不同，但是彼此之间又相互耦合影响。奖励设计的优劣直接影响学习到的策略的好坏_

3. **不稳定性：** 在多智能体系统中，多个智能体是同时学习的。当同伴的策略改变时，每个智能体自身的最优策略也可能会变化，这将对算法的**收敛性**带来影响。

4. **探索-利用：** 探索不光要考虑自身对环境的探索，也要对同伴的策略变化进行探索，可能打破同伴策略的平衡状态。每个智能体的探索都可能对同伴智能体的策略产生影响，这将使算法很难稳定，学习速度慢。

##### 🔹维度爆炸
解决方案:
1. 状态空间优化

    对于第一点，状态空间简化一般由**特征工程**来处理，一般方法:
    - PCA(Principal Component Analysis)常用于高维数据的降维
    - LDA(线性判别算法)算法

2. 联合动作空间优化
    - **Mean Field MARL算法:** 主要解决的是集中式多智能体强化学习中，$Q(s,a)$转换为只包含邻居之间相互作用的形式$$Q_{j}(s, a)=\frac{1}{N_{j}} \sum_{k \in N(j)} Q_{j}\left(s, a_{j}, a_{k}\right)$$其中$N(j)$表示智能体j邻居智能体的标签集<br>
    核心:近似保留邻居之间的交互，去掉了非邻居之间的交互
---
##### 🔹目标奖励函数
解决方案:
- **Curiosity Driven 好奇心驱动:** 

    使用内在奖励鼓励agent探索更陌生的状态，平衡探索与利用，本质上是提高了样本的利用效率，主要分为两类:
    - 基于状态计数<br>
    [Count-based exploration with neural density models](https://arxiv.org/pdf/1703.01310.pdf)<br>
    [Exploration: A study of count-based exploration for deep reinforcement learning](https://arxiv.org/abs/1611.04717)<br>
    - 基于状态预测误差的方法<br>
    [Incentivizing exploration in reinforcement learning with deep predictive models](https://arxiv.org/pdf/1507.00814v2)<br>
    [ICM（Curiosity-driven Exploration by Self- supervised Prediction）](https://arxiv.org/pdf/1705.05363.pdf)<br>

- **Hindsight Experience Replay（HER）无奖励样本学习:** 
    
    一般的强化学习方法对于无奖励的样本几乎没有利用，HER的思想就是从无奖励的样本中学习。HER建立在多目标强化学习的基础上，将失败的状态映射为新的目标<br>[HER文章](https://arxiv.org/abs/1707.01495)

- **Priority Experience Replay 采样率挂钩td_error:** 

    PER是DQN中最有效的改进手段之一，通过改变样本采样的概率来提高样本利用率和训练速度。Prioritized DQN中使用与td-error大小成正比的采样概率，在稀疏奖励中还可以对正样本采用更高的采样概率，防止在大量无奖励样本中“淹没”<br>[Prioritized DQN](https://arxiv.org/abs/1511.05952)

---
##### 🔹不稳定性
- **PPO:常用cliping截断**

    常用的方法类似于PPO的cliping截断--即为限制td_error的范围，非常好用，有相关论文 <br>[Deep Reinforcement Learning and the Deadly Triad](https://arxiv.org/abs/1812.02648)

- **DDPG: Soft target update 软更新**

    $$\omega^{-} \leftarrow \tau \omega+(1-\tau) \omega^{-}$$
    软更新让参数的更新不至于发生剧变，从而稳定了训练

- **DDQN: 使用目标网络**

    有优化操作会传播高估误差：DDQN（Double DQN）更加稳定，因为最优化操作会传播高估误差，所以同时训练两个Q network并选择较小的Q值用于计算TD-error，降低高估误差
    
- **Hierarchical Reinforcement Learning 分层强化学习**

    使用多层次的结构来学习不同层次的策略。把分层次解决复杂问题，把复杂劳动分解成多个简单劳动，使得算法更好收敛

----
##### 🔹探索-利用

- **在探索不足的情况下有鼓励探索的好奇心机制**

    [Exploration Strategies in Deep Reinforcement Learning](https://lilianweng.github.io/posts/2020-06-07-exploration-drl/)<br>
    [Curiosity-driven reinforcement learning with homeostatic regulation](https://arxiv.org/abs/1801.07440)

探索策略可以分为两类：有向的和无向的。

有向的探索策略考虑之前的历史，而无向的策略随机探索，不考虑之前的学习。这一节讨论三个部分:<br>
一是无向的探索，二是基于观测学习过程的有向探索，三是其他基于学习模型的有向探索

1. **Undirected Exploration 随机分布探索**

    动作通过随机分布生成，不考虑学习过程。Thrun认为无向的探索不如有向的探索。这里介绍三种无向探索策略：
    - 随机探索（random exploration）
    - 半均匀分布探索（semi-uniform distributed exploration）
        $$P(a)=\left\{\begin{array}{ll}P_{\text {best }}+\frac{1-P_{\text {best }}}{\# \text { of actions }} & , \text { if } a \text { maximizes } f(a) \\ \frac{1-P_{\text {best }}}{\# \text { of actions }} & , \text { otherwise }\end{array}\right.$$
        - 这种策略的基本思想是，给最佳的动作一个较高的概率，而其他动作则均匀地分配剩余的概率
        - 公式中的 $P_{\text{best}}$ 表示最佳动作的概率。当 $P_{\text{best}} = 1$ 时，策略完全利用已知的最佳动作；当 $P_{\text{best}} = 0$ 时，策略进行完全随机的探索。
    - 玻尔兹曼分布探索（Boltzmann distributed exploration） 
        $$P(a)=\frac{e^{f(a) \cdot \theta^{-1}}}{\sum_{a^{\prime} \in \text { Actions }} e^{f\left(a^{\prime}\right) \cdot \theta^{-1}}}$$
        - 这种策略基于动作的效用值（或回报）来分配概率。每个动作的概率与其效用值成指数比例。这里的 $\theta$ 是一个温度参数，用于控制探索和利用之间的权衡。
        - 当 $\theta \rightarrow 0$ 时，策略完全利用最佳动作；当 $\theta \rightarrow \infty$ 时，策略进行完全随机的探索
    随机探索效果不好，姑且不考虑，最终结论为：
    > 半均匀分布探索要比完全随机探索和玻尔兹曼分布探索都更好，半均匀分布探索策略直接为最佳动作分配一个预先定义的概率，而玻尔兹曼分布策略则基于动作的效用值来动态调整概率
    
2. **Directed Exploration 有向探索策略**
    
    在这里介绍四种基本的有向探索策略：
    - Counter-based exploration基于计数的探索
        $$\begin{array}{l}\text { eval }_{c}(a)=\alpha \cdot f(a)+\frac{c(s)}{E[c \mid s, a]} \\ E[c \mid s, a]=\sum P_{s \rightarrow s^{\prime}}(a) \cdot c\left(s^{\prime}\right)\end{array}$$
        动作用利用值和探索项的和来评估，这里的探索项是当前状态的计数和状态的期望计数的商
    - Counter-based exploration with decay基于计数的带衰减的探索
        $$c(s) \leftarrow \lambda \cdot c(s) \quad \forall s$$
        加了一个小于等于1的参数
    - Counter/Error-based exploration
        $$eval _{c}(a)=\alpha \cdot f(a)+\frac{c(s)}{E[c \mid s, a]}+\beta \cdot E\left[\Delta \hat{V}_{\text {last }} \mid s, a\right]$$
    - Recency-based exploration基于近因的探索
        $$\operatorname{eval}_{r}(a)=\alpha \cdot f(a)+\sqrt{E[\rho \mid s, a]}$$

3. **其他有向探索**
    - Competence Maps：使用一个辅助的数据结构来估计agent相信的程度，相信在哪个区域自己有足够的知识来做出好的决策。这个估计用来探索世界，选择的动作让agent去它认为有最小competence的区域。
    - Interval Estimation：学习的是动作价值以置信概率落在上下界中，但这种方法在实际中是有问题的，因为方法的复杂以及统计假设不能满足
    - Multiple-Hypothesis Heuristics：对Q-learning算法的一个修正，是计算可行的

---

##### 附录:详细讨论四种有向探索策略
1. **Counter-based exploration 基于计数的探索**

    这种策略主要依赖于为每一个状态-动作对 $(s, a)$ 计数。这个计数用于估计我们多少次访问了某一个状态并采取了某个动作。

    $$\text { eval }_{c}(a) = \alpha \cdot f(a) + \frac{c(s)}{E[c \mid s, a]}$$

    这里，$\alpha \cdot f(a)$ 是一个基于动作 $a$ 的价值函数。探索部分由 $\frac{c(s)}{E[c \mid s, a]}$ 表示，其中 $E[c \mid s, a]$ 是在状态 $s$ 并采取动作 $a$ 之后的预期计数。这种方法鼓励代理在未经常访问的状态中采取行动。

2. **Counter-based exploration with decay 基于计数的带衰减的探索**

    这种策略的思路是随着时间的推移逐渐减少状态的计数：

    $$c(s) \leftarrow \lambda \cdot c(s) \quad \forall s$$

    其中，$\lambda$ 是一个衰减因子，它通常小于等于1。这意味着早期的访问将逐渐失去其重要性，使得代理有更大的倾向去重新探索那些它很久没有访问的状态。

3. **Counter/Error-based exploration**

    这种策略不仅考虑访问计数，还考虑价值函数的估计误差。当我们对某个状态-动作对的价值函数的估计越不确定时，我们应该更加倾向于探索这个状态-动作对。

    $$\text{eval}_{c}(a) = \alpha \cdot f(a) + \frac{c(s)}{E[c \mid s, a]} + \beta \cdot E\left[\Delta \hat{V}_{\text {last }} \mid s, a\right]$$

    其中，$E\left[\Delta \hat{V}_{\text {last }} \mid s, a\right]$ 表示在状态 $s$ 采取动作 $a$ 后价值函数的预期变化。

4. **Recency-based exploration 基于近因的探索**

    这种策略的核心思想是优先考虑那些最近没有访问过的状态-动作对。其探索度量基于最后一次访问该状态-动作对的时间。

    $$\text{eval}_{r}(a) = \alpha \cdot f(a) + \sqrt{E[\rho \mid s, a]}$$

    其中，$E[\rho \mid s, a]$ 表示自从上次访问状态 $s$ 并采取动作 $a$ 以来的时间。

---